# Pandas

This notebook presents an introduction to the `pandas` package.
We focus on only the most commonly used features of `pandas` as they pertain to geospatial data analysis. More extensive references are given at the end of the notebook.

In [ ]:
import pandas
import numpy as np

In [ ]:
ward = np.tile([1,2,3,4,5], 5)
population = np.random.randint(5000, size=(25,))
poverty = np.random.random(size=(25,)) * .4

In [ ]:
poverty

In [ ]:
population

In [ ]:
ward

In [ ]:
df = pandas.DataFrame({'population': population,
                      'ward': ward,
                      'poverty': poverty})

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
type(df)

In [ ]:
type(df.population)

## indexing

In [ ]:
df['population']

In [ ]:
df.population

In [ ]:
df[0:4]

In [ ]:
df[-4:]

In [ ]:
df[df.ward==2]

In [ ]:
df[(df.ward==2) & (df.population < 1000)]

In [ ]:
df[~(df.ward==2) & (df.population < 1000)] # not in ward 2 and less than 1000 population

In [ ]:
df[~((df.ward==2) & (df.population < 1000))] # not (in ward 2 and less than 1000 population)

In [ ]:
df[(df.ward==2) | (df.population < 1000)] #in ward 2 or less than 1000 population)

## New Columns

In [ ]:
df.head()

In [ ]:
pop_pov = df.population * df.poverty
pop_pov

In [ ]:
df['pop_pov'] = pop_pov.astype('int')

In [ ]:
df.head()

## Aggregation/Groupby

In [ ]:
df.groupby(by='ward').sum()

In [ ]:
df.groupby(by='ward').sum()[['population', 'pop_pov']]

In [ ]:
ward_df = df.groupby(by='ward').sum()[['population', 'pop_pov']]

In [ ]:
ward_df['poverty'] = ward_df.pop_pov / ward_df.population

In [ ]:
ward_df.head()

## Joins

In [ ]:
ward_df.index

In [ ]:
df.merge(ward_df, how='left', on='ward')

In [ ]:
df = df.merge(ward_df, how='left', on='ward')

In [ ]:
names = df.columns

In [ ]:
new_names = [name.replace("_y", "_ward") for name in names]
df.columns = new_names
df.head()

In [ ]:
df[df.poverty_x > df.poverty_ward]

<div class="alert alert-success" style="font-size:120%">
<b>Exercise</b>: <br>
Which ward has the highest poverty rate?
</div>

In [ ]:
# %load solutions/120_1.py

<div class="alert alert-success" style="font-size:120%">
<b>Exercise</b>: <br>
Which tract the highest poverty rate as a percentage of its ward's poverty rate?
</div>

In [ ]:
# %load solutions/120_2.py

## Dealing with *real data*

In [ ]:
data = pandas.read_csv('data/CAINC1__ALL_STATES_1969_2017.csv', encoding='latin-1', 
                      skipfooter=3, engine='python')

In [ ]:
data.shape

In [ ]:
data.head()

In [ ]:
pandas.set_option('display.max_columns', 500)

In [ ]:
data.head()

In [ ]:
data.tail(10)

In [ ]:
data.columns

In [ ]:
data['1969']

In [ ]:
data['1969'].sort_values()

so there are undisclosed values in the cases of `(NA)`

In [ ]:
data.isna().sum() # so no na values in the numpy / pandas sense

In [ ]:
data1 = data.replace("(NA)", 0)
data1['1969'] = data1['1969'].astype(int)

In [ ]:
data1['1969'].sort_values()

In [ ]:
for year in range(1969, 2018):
    print(year, data[data[str(year)]=='(NA)'].shape)

## subsetting

In [ ]:
small = data[data.LineCode.isin( [2, 3] )]

In [ ]:
small.shape

In [ ]:
data.shape

In [ ]:
small.head()

In [ ]:
for year in range(1969, 2018):
    small = small[small[str(year)] != "(NA)"] #drop all records with NA

In [ ]:
small.shape

In [ ]:
small.head(20)

In [ ]:
small['1969']

In [ ]:
convert_dict = dict([(str(year), int) for year in range (1969, 2018)])

In [ ]:
small = small.astype(convert_dict)

In [ ]:
small['1969']

In [ ]:
small['2017']

In [ ]:
geofips = pandas.unique(small.GeoFIPS)

In [ ]:
geofips

In [ ]:
geofips.shape

In [ ]:
small['GeoFIPS'] = [fips.replace("\"", "").strip() for fips in small.GeoFIPS]

In [ ]:
geofips = pandas.unique(small.GeoFIPS)

In [ ]:
geofips

In [ ]:
pc_inc = small[small.LineCode==3]

In [ ]:
pc_inc.shape

In [ ]:
pc_inc.head()

In [ ]:
pc_inc.max()

In [ ]:
cid_1969 = pc_inc.columns.get_loc('1969')
cid_1969

In [ ]:
pc_inc.iloc[:,8]

In [ ]:
pc_inc.iloc[:, 8:20]

In [ ]:
pc_inc.iloc[:, 8:].idxmax(axis=0) 

In [ ]:
max_ids = pc_inc.iloc[:, 8:].idxmax() 
pc_inc.loc[max_ids]

In [ ]:
for y, max_id in enumerate(max_ids):
    year = y + 1969
    name = pc_inc.loc[max_id].GeoName
    pci = pc_inc.loc[max_id, str(year)]
    print(year, pci, name)
    

<div class="alert alert-success" style="font-size:120%">
<b>Exercise</b>: <br>
Identify the area with the lowest per-capita income each year.
</div>

In [ ]:
min_ids = pc_inc.iloc[:, 8:].idxmin() 
for y, min_id in enumerate(min_ids):
    year = y + 1969
    name = pc_inc.loc[min_id].GeoName
    pci = pc_inc.loc[min_id, str(year)]
    print(year, pci, name)

<div class="alert alert-success" style="font-size:120%">
<b>Exercise</b>: <br>
    As a percentage of the minimum per-captia income, calculate the relative income gap between the extremes of the income distribution each year.
    
   Identify the year with the maximum relative income gap.
   
</div>

In [ ]:
# %load solutions/120_3.py

<div class="alert alert-info" style="font-size:120%">
<b>Further Readings</b>: <br>
    <a rel='further' href="https://pandas.pydata.org/pandas-docs/stable/index.html">Pandas Documentation</a>
</div>

---

<a rel="license" href="http://creativecommons.org/licenses/by-nc-
sa/4.0/"><img alt="Creative Commons License" style="border-width:0"
src="https://i.creativecommons.org/l/by-nc-sa/4.0/88x31.png" /></a><br /><span
xmlns:dct="http://purl.org/dc/terms/" property="dct:title">Pandas</span> by <a xmlns:cc="http://creativecommons.org/ns#"
href="http://sergerey.org" property="cc:attributionName"
rel="cc:attributionURL">Serge Rey</a> is licensed under a <a
rel="license" href="http://creativecommons.org/licenses/by-nc-sa/4.0/">Creative
Commons Attribution-NonCommercial-ShareAlike 4.0 International License</a>.